In [8]:
from playwright.async_api import async_playwright
import pandas as pd
from bs4 import BeautifulSoup
import requests
import datetime
import time
import json
import csv
import numpy as np
import re
import lxml.html

In [9]:
playwright = await async_playwright().start()

In [10]:
browser = await playwright.chromium.launch(headless = False)

In [11]:
page = await browser.new_page()

In [12]:
await page.goto("https://awt.cbp.gov/")
time.sleep(6)

### Choose an airport -- JFK in our case: 

In [13]:
await page.get_by_label("Select Your Airport/Terminal").select_option("JFK")

['JFK']

In [14]:
await page.get_by_role("button", name="Create Report").click()
time.sleep(10)

In [52]:
page = await page.content()
tables = pd.read_html(page)
tables[0]

In [16]:
table = tables[0]

In [17]:
table.columns.values

array([('Airport', 'Airport', 'Airport'),
       ('Terminal', 'Terminal', 'Terminal'), ('Date', 'Date', 'Date'),
       ('Hour', 'Hour', 'Hour'),
       ('U.S. Citizen', 'U.S. Citizen', 'Average Wait  Time'),
       ('U.S. Citizen', 'U.S. Citizen', 'Max Wait  Time'),
       ('Non U.S. Citizen', 'Non U.S. Citizen', 'Average Wait  Time'),
       ('Non U.S. Citizen', 'Non U.S. Citizen', 'Max Wait  Time'),
       ('All', 'Wait Times', 'Average Wait  Time'),
       ('All', 'Wait Times', 'Max Wait  Time'),
       ('All', 'Number Of Passengers Time Interval', '0-15'),
       ('All', 'Number Of Passengers Time Interval', '16-30'),
       ('All', 'Number Of Passengers Time Interval', '31-45'),
       ('All', 'Number Of Passengers Time Interval', '46-60'),
       ('All', 'Number Of Passengers Time Interval', '61-90'),
       ('All', 'Number Of Passengers Time Interval', '91-120'),
       ('All', 'Number Of Passengers Time Interval', '120 plus'),
       ('All', 'Unnamed: 17_level_1', 'Excluded'),

### We clean up the columns and consolidate in one row

In [18]:
table.columns = ['_'.join(col) for col in table.columns.values]

In [19]:
table.rename(columns = {'Airport_Airport_Airport': 'Airport', 'Terminal_Terminal_Terminal':'Terminal', 'Date_Date_Date':'date', 'Hour_Hour_Hour':'hour' }, inplace = True)

In [20]:
table.rename(columns = {'U.S. Citizen_U.S. Citizen_Average Wait  Time': 'US Citizen Avg', 'U.S. Citizen_U.S. Citizen_Max Wait  Time':'US Citizen Max', 'Non U.S. Citizen_Non U.S. Citizen_Average Wait  Time':'Non-Citizen Avg', 'Non U.S. Citizen_Non U.S. Citizen_Max Wait  Time': 'Non-Citizen Max' }, inplace = True)

In [21]:
table.rename(columns = {'All_Wait Times_Average Wait  Time': 'Avg Wait Time', 'All_Wait Times_Max Wait  Time':'Max Wait Time', 'All_Unnamed: 18_level_1_Total':'Total Passengers', 'All_Unnamed: 19_level_1_Flights': 'Total Flights' }, inplace = True)

In [22]:
table = table[['Airport', 'Terminal', 'date', 'hour','US Citizen Avg', 'US Citizen Max', 'Non-Citizen Avg','Non-Citizen Max','Avg Wait Time', 'Max Wait Time','Total Passengers','Total Flights' ]]

In [26]:
term1 = table[table["Terminal"].str.contains("1")].reset_index(inplace=False)

In [53]:
term1 = term1.drop('index', axis =1)

In [28]:
reduced1 = term1[['hour','US Citizen Avg', 'Non-Citizen Avg']]

In [54]:
reducedtransposed1 = reduced1.transpose()

In [31]:
reducedtransposed1.columns = reducedtransposed1.iloc[0]

In [32]:
reducedtransposed1 = reducedtransposed1.drop(index= "hour")

In [33]:
reducedtransposed1 

hour,0900 - 1000,1100 - 1200,1200 - 1300,1300 - 1400,1400 - 1500,1500 - 1600,1600 - 1700,1700 - 1800,1800 - 1900,1900 - 2000,2000 - 2100,2100 - 2200,2200 - 2300,2300 - 0000
US Citizen Avg,31,38,41,27,53,49,44,21,28,25,26,33,31,17
Non-Citizen Avg,41,86,58,77,100,114,110,98,47,34,66,65,36,17


In [34]:
reducedtransposed1.to_csv("term1_us_nonus.csv")

### Terminal 4:

In [36]:
term4 = table[table["Terminal"].str.contains("4")].reset_index(inplace=False)

In [37]:
term4 = term4.drop('index', axis =1)
reduced4 = term4[['hour','US Citizen Avg', 'Non-Citizen Avg']]
reducedtransposed4 = reduced4.transpose()
reducedtransposed4.columns = reducedtransposed4.iloc[0]
reducedtransposed4 = reducedtransposed4.drop(index= "hour")

In [43]:
reducedtransposed4.to_csv("term4_us_nonus.csv")

### Terminal 5: 

In [40]:
term5 = table[table["Terminal"].str.contains("5")].reset_index(inplace=False)

In [41]:
term5 = term5.drop('index', axis =1)
reduced5 = term5[['hour','US Citizen Avg', 'Non-Citizen Avg']]
reducedtransposed5 = reduced5.transpose()
reducedtransposed5.columns = reducedtransposed5.iloc[0]
reducedtransposed5 = reducedtransposed5.drop(index= "hour")

In [45]:
reducedtransposed5.to_csv("term5_us_nonus.csv")

### Terminal 7: 

In [47]:
term7 = table[table["Terminal"].str.contains("7")].reset_index(inplace=False)

In [48]:
term7 = term7.drop('index', axis =1)
reduced7 = term7[['hour','US Citizen Avg', 'Non-Citizen Avg']]
reducedtransposed7 = reduced7.transpose()
reducedtransposed7.columns = reducedtransposed7.iloc[0]
reducedtransposed7 = reducedtransposed7.drop(index= "hour")

In [49]:
reducedtransposed7.to_csv("term7_us_nonus.csv")

### Terminal 8: 

In [55]:
term8 = table[table["Terminal"].str.contains("8")].reset_index(inplace=False)

In [56]:
reduced8 = term8[['hour','US Citizen Avg', 'Non-Citizen Avg']]
reducedtransposed8 = reduced8.transpose()
reducedtransposed8.columns = reducedtransposed8.iloc[0]
reducedtransposed8 = reducedtransposed8.drop(index= "hour")

In [57]:
reducedtransposed8.to_csv("term8_us_nonus.csv")

In [58]:
ls

README.md              original_cbp.ipynb     term7_us_nonus.csv
action.yml             term1_us_nonus.csv     term8_us_nonus.csv
cbp1.ipynb             term4_us_nonus.csv
jfkterm8_us_nonus.csv  term5_us_nonus.csv


In [59]:
await browser.close()